# 1.0 Importando Bibliotecas

In [139]:
# Install Plotly
!pip install Plotly==4.12

# Install Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

In [140]:
import os.path
import sys, json
import requests
import subprocess

import numpy as np
import pandas as pd
import plotly.express as px

from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

# 2.0 Configurando o Ndrok

In [141]:
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [142]:
Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

# 3.0 O Dash apps

Visualizando os dados do nosso dataframe

In [143]:
import pandas as pd

dadosInpe48h = 'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?id=focos_brasil&time=48h' 

df = pd.read_csv(dadosInpe48h)

df.head()

,FID,id,longitude,latitude,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,geom
0,focos_brasil_ontem_hoje_ref.9f5c6bfd-71ec-3e24...,9f5c6bfd-71ec-3e24-b168-511892e85b5d,-56.515,-15.470,2020-11-22T17:50:00,AQUA_M-T,JANGADA,MATO GROSSO,Brasil,5104906,51,33,NaN,NaN,NaN,Cerrado,POINT (-15.47 -56.515)
1,focos_brasil_ontem_hoje_ref.045060ba-52df-3568...,045060ba-52df-3568-a0ee-0a103eeca33e,-65.180,-10.687,2020-11-22T17:50:00,AQUA_M-T,GUAJARÁ-MIRIM,RONDÔNIA,Brasil,1100106,11,33,NaN,NaN,NaN,Amazônia,POINT (-10.687 -65.18)
2,focos_brasil_ontem_hoje_ref.aea5edc1-7c10-31a6...,aea5edc1-7c10-31a6-80c2-09d39e97a95d,-65.075,-10.093,2020-11-22T17:50:00,AQUA_M-T,NOVA MAMORÉ,RONDÔNIA,Brasil,1100338,11,33,NaN,NaN,NaN,Amazônia,POINT (-10.093 -65.075)
3,focos_brasil_ontem_hoje_ref.6135b6b0-38fd-3183...,6135b6b0-38fd-3183-bcfe-4138d1520ff4,-59.926,-14.378,2020-11-22T17:50:00,AQUA_M-T,NOVA LACERDA,MATO GROSSO,Brasil,5106182,51,33,NaN,NaN,NaN,Amazônia,POINT (-14.378 -59.926)
4,focos_brasil_ontem_hoje_ref.f4721931-4351-35e9...,f4721931-4351-35e9-a350-861a674ca4e9,-66.637,-8.878,2020-11-22T17:50:00,AQUA_M-T,LÁBREA,AMAZONAS,Brasil,1302405,13,33,NaN,NaN,NaN,Amazônia,POINT (-8.878 -66.637)


Código da aplicação com o Dash app

In [144]:
%%writefile my_dash_app.py
import dash
import plotly.graph_objs as go
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd 

token = open("/content/token.txt").read()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

dadosInpe48h = 'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?id=focos_brasil&time=48h' 

df = pd.read_csv(dadosInpe48h)

fig = px.scatter_mapbox(df, 
                        lat='latitude', 
                        lon='longitude',
                        color='bioma',
                       
                        zoom=3.8,
                        hover_data=['municipio', 'estado'],
                        mapbox_style='carto-positron')

fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token,)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, paper_bgcolor="#1C1C1C", font={"color":"White"})


fig2 = px.histogram (df,
                     x = 'estado',
                     color = 'bioma',
                     
                     title='Os tipos de biomas',
                     labels={'estado':'Estados'},
                     )

fig2.update_layout(paper_bgcolor="#1C1C1C", font={"color":"White"}, plot_bgcolor='#363636')



app.layout = html.Div( children=[
    dcc.Graph( 
        id='life-exp-vs-gd',
        figure=fig
    ),  html.Div(
                children=[],
              ),
              dcc.Graph(id="histograma", figure=fig2),
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Overwriting my_dash_app.py


Run the dash app:

In [145]:
download_ngrok()

In [147]:
retorno = get_tunnel()
print(retorno)
!python my_dash_app.py

Response(url='https://724f059561b9.ngrok.io', error=None)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "my_dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
^C


In case you want to build a dash with user interaction, use [Dash Callbacks](https://dash.plotly.com/basic-callbacks).

# Reference

* [Video: Tableros Dash en Colab y tuneles con ngrok](https://www.youtube.com/watch?v=g6M3mAHFcyU)
*   [ngrok](https://ngrok.com/)
*   [Dash Plotly](https://dash.plotly.com/)
*   [Interactive Graphing](https://dash.plotly.com/interactive-graphing)
*   [Dash for Beginners - DataCamp](https://www.datacamp.com/community/tutorials/learn-build-dash-python)
